In [0]:
import datetime

In [0]:
if not any(mount.mountPoint == "/mnt/azuremountcfgc" for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = "wasbs://gccontainer@cfgcstorage.blob.core.windows.net",
        mount_point = "/mnt/azuremountcfgc",
        extra_configs = {"fs.azure.account.key.cfgcstorage.blob.core.windows.net":"yyHYfFxpI0AWxsOdFJH699cGYQh60CutcVcsrJfo5UMPixDJdcRTNSudo+Nfz9/lrMZTxyAjNiZO+AStaG7M1w=="}
)

In [0]:
trade_date = datetime.datetime.strptime('2020-08-06', '%Y-%m-%d')
prev_date_str = (trade_date - datetime.timedelta(1)).strftime('%Y-%m-%d')
df = spark.read.parquet(f"/mnt/azuremountcfgc/trade/trade_dt={trade_date.strftime('%Y-%m-%d')}")

In [0]:
df = spark.read.parquet("/mnt/azuremountcfgc/trade/*/*")

In [0]:
df.show()

+----------+------+--------+--------------------+------------+--------+-------------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|trade_pr|    max(arrival_tm)|
+----------+------+--------+--------------------+------------+--------+-------------------+
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 18:01:...|          70|      77|2020-08-06 09:30:00|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 13:09:...|          30|      78|2020-08-06 09:30:00|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 10:42:...|          10|      79|2020-08-06 09:30:00|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 19:09:...|          80|      79|2020-08-06 09:30:00|
|2020-08-06|  SYMB|  NASDAQ|2020-08-06 14:15:...|          40|      32|2020-08-06 09:30:00|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 15:39:...|          50|      78|2020-08-06 09:30:00|
|2020-08-06|  SYMB|  NASDAQ|2020-08-06 10:47:...|          10|      32|2020-08-06 09:30:00|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 20:23:...|          90|      78|2020-08-0

In [0]:
df.createOrReplaceTempView("all_trades")

In [0]:
df2 = spark.sql("select symbol, event_tm, event_seq_nb, trade_pr, exchange, trade_dt from all_trades where trade_dt = '2020-08-06'")

In [0]:
df2.show()

+------+--------------------+------------+--------+--------+----------+
|symbol|            event_tm|event_seq_nb|trade_pr|exchange|  trade_dt|
+------+--------------------+------------+--------+--------+----------+
|  SYMA|2020-08-06 18:01:...|          70|      77|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 13:09:...|          30|      78|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 10:42:...|          10|      79|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 19:09:...|          80|      79|  NASDAQ|2020-08-06|
|  SYMB|2020-08-06 14:15:...|          40|      32|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 15:39:...|          50|      78|  NASDAQ|2020-08-06|
|  SYMB|2020-08-06 10:47:...|          10|      32|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 20:23:...|          90|      78|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 16:43:...|          60|      78|  NASDAQ|2020-08-06|
|  SYMA|2020-08-06 12:00:...|          20|      77|  NASDAQ|2020-08-06|
|  SYMB|2020-08-06 15:20:...|          50|      33|  NASDAQ|2020

In [0]:
df2.createOrReplaceTempView("trades_20200806")

In [0]:
mov_avg_df = spark.sql("""
    SELECT
        symbol
        , exchange
        , trade_dt
        , event_tm
        , event_seq_nb
        , trade_pr
        , AVG(trade_pr) OVER (PARTITION BY symbol, exchange ORDER BY event_tm DESC RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr
    FROM
        trades_20200806
""")

In [0]:
mov_avg_df.show(10, False)

+------+--------+----------+-----------------------+------------+--------+----------+
|symbol|exchange|trade_dt  |event_tm               |event_seq_nb|trade_pr|mov_avg_pr|
+------+--------+----------+-----------------------+------------+--------+----------+
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 21:33:38.812|100         |77      |77.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 20:23:01.854|90          |78      |78.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 19:09:38.472|80          |79      |79.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 18:01:33.234|70          |77      |77.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 16:43:23.372|60          |78      |78.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 15:39:00.929|50          |78      |78.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 14:27:08.62 |40          |76      |76.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 13:09:29.883|30          |78      |78.0000   |
|SYMA  |NASDAQ  |2020-08-06|2020-08-06 12:00:29.595|20

In [0]:
mov_avg_df.write.option("overwriteSchema", "true").mode("overwrite").saveAsTable("temp_trade_moving_avg")

In [0]:
df3 = spark.sql("""
    SELECT
        symbol
        , event_tm
        , event_seq_nb
        , trade_pr
        , exchange
        , trade_dt
    FROM
        all_trades
    WHERE
        trade_dt = '2020-08-05'
""")

In [0]:
df3.show()

+------+--------------------+------------+--------+--------+----------+
|symbol|            event_tm|event_seq_nb|trade_pr|exchange|  trade_dt|
+------+--------------------+------------+--------+--------+----------+
|  SYMC|2020-08-05 21:49:...|         100|     158|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 14:36:...|          40|     159|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 17:02:...|          60|     160|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 19:19:...|          80|     158|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 10:47:...|          10|     160|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 20:33:...|          90|     157|  NASDAQ|2020-08-05|
|  SYMB|2020-08-05 17:42:...|          70|      36|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 12:00:...|          20|     160|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 18:15:...|          70|     158|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 15:49:...|          50|     157|  NASDAQ|2020-08-05|
|  SYMC|2020-08-05 13:13:...|          30|     158|  NASDAQ|2020

In [0]:
df3.createOrReplaceTempView("trades_20200805")

In [0]:
last_pr_df = spark.sql("""
    SELECT
        symbol
        , exchange
        , last_pr
    FROM
        (
        SELECT
            symbol
            , exchange
            , event_tm
            , event_seq_nb
            , trade_pr
            , AVG(trade_pr) OVER (PARTITION BY symbol, exchange ORDER BY event_tm DESC RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) AS last_pr
            , RANK() OVER (PARTITION BY symbol, exchange ORDER BY event_tm DESC) AS rank
        FROM
            trades_20200805
        )
    WHERE
        rank = 1
""")

In [0]:
last_pr_df.show(10, False)

+------+--------+--------+
|symbol|exchange|last_pr |
+------+--------+--------+
|SYMA  |NASDAQ  |77.0000 |
|SYMA  |NYSE    |78.0000 |
|SYMB  |NASDAQ  |36.0000 |
|SYMB  |NYSE    |34.0000 |
|SYMC  |NASDAQ  |158.0000|
|SYMC  |NYSE    |161.0000|
+------+--------+--------+



In [0]:
quote_df = spark.read.parquet("/mnt/azuremountcfgc/quote/*/*")

In [0]:
quote_df.createOrReplaceTempView("all_quotes")

In [0]:
quote_20200806 = spark.sql("""
    SELECT
        symbol
        , exchange
        , trade_dt
        , event_tm
        , event_seq_nb
        , bid_pr
        , bid_size
        , ask_pr
        , ask_size
    FROM
        all_quotes
    WHERE
        trade_dt = '2020-08-06'
""")

In [0]:
quote_20200806.show(10, False)

+------+--------+----------+-----------------------+------------+------+--------+------+--------+
|symbol|exchange|trade_dt  |event_tm               |event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|
+------+--------+----------+-----------------------+------------+------+--------+------+--------+
|SYMA  |NYSE    |2020-08-06|2020-08-06 09:39:01.293|1           |78    |100     |78    |100     |
|SYMA  |NYSE    |2020-08-06|2020-08-06 13:54:59.362|36          |78    |100     |79    |100     |
|SYMB  |NYSE    |2020-08-06|2020-08-06 12:59:00.012|28          |33    |100     |34    |100     |
|SYMB  |NYSE    |2020-08-06|2020-08-06 11:16:52.626|14          |32    |100     |34    |100     |
|SYMA  |NYSE    |2020-08-06|2020-08-06 18:01:15.431|68          |79    |100     |81    |100     |
|SYMA  |NYSE    |2020-08-06|2020-08-06 15:17:45.541|47          |77    |100     |79    |100     |
|SYMB  |NYSE    |2020-08-06|2020-08-06 13:34:44.666|33          |32    |100     |33    |100     |
|SYMA  |NYSE    |202

In [0]:
quote_20200806.createOrReplaceTempView("quotes_20200806")

In [0]:
quote_union_temp_avg = spark.sql("""
    SELECT
        trade_dt
        , "Q" as rec_type
        , symbol
        , event_tm
        , event_seq_nb
        , exchange
        , bid_pr
        , bid_size
        , ask_pr
        , ask_size
        , NULL AS trade_pr
        , NULL AS mov_avg_pr
    FROM
        quotes_20200806
    UNION
    SELECT
        trade_dt
        , "T" as rec_type
        , symbol
        , event_tm
        , event_seq_nb
        , exchange
        , NULL AS bid_pr
        , NULL AS bid_size
        , NULL AS ask_pr
        , NULL AS ask_size
        , trade_pr
        , mov_avg_pr
    FROM
        temp_trade_moving_avg
""")

In [0]:
quote_union_temp_avg.show(20, False)

+----------+--------+------+-----------------------+------------+--------+------+--------+------+--------+--------+----------+
|trade_dt  |rec_type|symbol|event_tm               |event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+--------+------+-----------------------+------------+--------+------+--------+------+--------+--------+----------+
|2020-08-06|Q       |SYMA  |2020-08-06 19:43:42.686|82          |NYSE    |77    |100     |78    |100     |null    |null      |
|2020-08-06|Q       |SYMB  |2020-08-06 15:03:15.079|44          |NYSE    |34    |100     |34    |100     |null    |null      |
|2020-08-06|Q       |SYMA  |2020-08-06 15:26:00.294|48          |NYSE    |75    |100     |76    |100     |null    |null      |
|2020-08-06|Q       |SYMA  |2020-08-06 18:37:03.342|73          |NYSE    |75    |100     |77    |100     |null    |null      |
|2020-08-06|Q       |SYMA  |2020-08-06 12:09:41.326|21          |NYSE    |75    |100     |76    |100     |null 

In [0]:
quote_union_temp_avg.createOrReplaceTempView("quote_union")

In [0]:
quote_union_update = spark.sql("""
    SELECT
        *
    FROM
        (
        SELECT
            symbol
            , exchange
            , trade_pr as last_trade_pr
            , trade_dt
            , event_tm
            , event_seq_nb
            , bid_pr
            , bid_size
            , ask_pr
            , ask_size
            , trade_pr
            , mov_avg_pr as last_mov_avg_pr
            , rec_type
            , RANK() OVER(PARTITION BY symbol, exchange ORDER BY event_tm DESC) AS rank
        FROM
            quote_union
        WHERE
            rec_type = 'T'
        )
    WHERE
        rank = 1
""")

In [0]:
quote_union_update = spark.sql("""
    SELECT
        *
        , last_trade_pr
        , last_mov_avg_pr
    FROM (
        SELECT 
            q2.*
            , row_number() OVER (PARTITION BY q2.symbol, q2.exchange, q2.event_seq_nb ORDER BY q1.event_tm desc) as row_num
            , avg(q1.trade_pr) over (partition by q2.symbol, q2.exchange order by q1.event_tm range between interval 30 minutes preceding and current row) as last_mov_avg_pr
            , q1.trade_pr as last_trade_pr
        from quote_union q1
        right join quote_union q2
            on q1.symbol = q2.symbol
            and q1.exchange = q2.exchange
            and q1.event_tm < q2.event_tm
            and q1.rec_type = 'T'
    )
    where row_num = 1 
    and rec_type = 'Q'
    and trade_dt = '2020-08-06'
    order by symbol, event_tm, event_seq_nb
    """)

In [0]:
quote_union_update.show(20, False)

+----------+--------+------+-----------------------+------------+--------+------+--------+------+--------+--------+----------+-------+---------------+-------------+-------------+---------------+
|trade_dt  |rec_type|symbol|event_tm               |event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|row_num|last_mov_avg_pr|last_trade_pr|last_trade_pr|last_mov_avg_pr|
+----------+--------+------+-----------------------+------------+--------+------+--------+------+--------+--------+----------+-------+---------------+-------------+-------------+---------------+
|2020-08-06|Q       |SYMA  |2020-08-06 09:38:08.093|1           |NASDAQ  |78    |100     |80    |100     |null    |null      |1      |null           |null         |null         |null           |
|2020-08-06|Q       |SYMA  |2020-08-06 09:39:01.293|1           |NYSE    |78    |100     |78    |100     |null    |null      |1      |null           |null         |null         |null           |
|2020-08-06|Q       |SYMA

In [0]:
quote_union_update.createOrReplaceTempView("quote_union_update")

In [0]:
quote_update = spark.sql("""
    SELECT
        q.trade_dt
        , q.symbol
        , q.event_tm
        , q.event_seq_nb
        , q.exchange
        , q.bid_pr
        , q.bid_size
        , q.ask_pr
        , q.ask_size
        , q2.last_trade_pr
        , q2.last_mov_avg_pr
    FROM
        quote_union as q
    LEFT JOIN
        quote_union_update as q2
    ON
        q.exchange = q2.exchange
    AND
        q.symbol = q2.symbol
    WHERE
        q.rec_type = "Q"
    ORDER BY
        q.symbol, q.event_tm, q.event_seq_nb
""")

In [0]:
quote_update.show(10, False)

+----------+------+-----------------------+------------+--------+------+--------+------+--------+-------------+---------------+
|trade_dt  |symbol|event_tm               |event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|
+----------+------+-----------------------+------------+--------+------+--------+------+--------+-------------+---------------+
|2020-08-06|SYMA  |2020-08-06 09:38:08.093|1           |NASDAQ  |78    |100     |80    |100     |77           |77.0000        |
|2020-08-06|SYMA  |2020-08-06 09:39:01.293|1           |NYSE    |78    |100     |78    |100     |76           |76.0000        |
|2020-08-06|SYMA  |2020-08-06 09:46:05.163|2           |NASDAQ  |77    |100     |77    |100     |77           |77.0000        |
|2020-08-06|SYMA  |2020-08-06 09:47:20.398|2           |NYSE    |77    |100     |77    |100     |76           |76.0000        |
|2020-08-06|SYMA  |2020-08-06 09:52:14.798|3           |NASDAQ  |79    |100     |79    |100     |77     

In [0]:
quote_update.createOrReplaceTempView("quote_update")

In [0]:
quote_final = spark.sql("""
    SELECT
        q.*
        , q.bid_pr - t.trade_pr as bid_pr_mv
        , q.ask_pr - t.trade_pr as ask_pr_mv
    FROM
        quote_update as q
    LEFT OUTER JOIN
        trades_20200805 as t
    ON
        q.exchange = t.exchange
    AND
        q.symbol = t.symbol
""")

In [0]:
quote_final.show()

+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|  trade_dt|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|bid_pr_mv|ask_pr_mv|
+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|2020-08-06|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|           77|        77.0000|        1|        3|
|2020-08-06|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|           77|        77.0000|        0|        2|
|2020-08-06|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|           77|        77.0000|        1|        3|
|2020-08-06|  SYMA|2020-08-06 09:38:...|           1|  NASDAQ|    78|     100|    80|     100|           77|        77.0000|      

In [0]:
trade_date = trade_date.strftime('%Y-%m-%d')
quote_final.write.mode("overwrite").parquet(f"/mnt/azuremountcfgc/quote_trade_analytical/date={trade_date}")